In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import scipy.stats as stats
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# LightGBM
import lightgbm as lgb
import sys

import category_encoders as ce

import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from plotly import tools

import warnings
warnings.filterwarnings('ignore')

from ipywidgets import Select,interact, interactive, fixed, interact_manual

from matplotlib.font_manager import FontProperties

In [2]:
#fp = FontProperties(fname=r'C:\WINDOWS\Fonts\meiryob.ttc', size=16)

df = pd.read_csv('deaths_cumulative_daily.csv')
df_copy = df.copy()

df_copy.head()

,Date,Prefecture,Deaths(Cumulative)
0,2020/5/9,ALL,613
1,2020/5/9,Hokkaido,48
2,2020/5/9,Aomori,0
3,2020/5/9,Iwate,0
4,2020/5/9,Miyagi,1


In [3]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25392 entries, 0 to 25391
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Date                25392 non-null  object
 1   Prefecture          25392 non-null  object
 2   Deaths(Cumulative)  25392 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 595.2+ KB


In [4]:
df_2 = pd.read_csv('newly_confirmed_cases_daily.csv')
df_2_copy = df_2.copy()

ALL = df_2_copy[df_2_copy.iloc[:, 1] == "ALL"].index
df_2_copy = df_2_copy.drop(df_2_copy.index[ALL] , axis=0)

df_2_copy.head()

,Date,Prefecture,Newly confirmed cases
1,2020/1/16,Hokkaido,0
2,2020/1/16,Aomori,0
3,2020/1/16,Iwate,0
4,2020/1/16,Miyagi,0
5,2020/1/16,Akita,0


In [5]:
df_3 = pd.read_csv('pcr_tested_daily.csv')
df_3_copy = df_3.copy()

df_3_copy = df_3_copy.rename(columns = {'日付':'Date'})

df_3_copy

,Date,PCR 検査実施件数(単日)
0,2020/2/5,4.0
1,2020/2/6,19.0
2,2020/2/7,9.0
3,2020/2/8,4.0
4,2020/2/9,10.0
...,...,...
618,2021/10/15,64812.0
619,2021/10/16,42154.0
620,2021/10/17,18498.0
621,2021/10/18,51790.0


In [6]:
ALL = df_copy[df_copy.iloc[:, 1] == "ALL"].index
df_copy = df_copy.drop(df_copy.index[ALL] , axis=0)
df_copy

,Date,Prefecture,Deaths(Cumulative)
1,2020/5/9,Hokkaido,48
2,2020/5/9,Aomori,0
3,2020/5/9,Iwate,0
4,2020/5/9,Miyagi,1
5,2020/5/9,Akita,0
...,...,...,...
25387,2021/10/19,Kumamoto,136
25388,2021/10/19,Oita,84
25389,2021/10/19,Miyazaki,41
25390,2021/10/19,Kagoshima,65


In [7]:
df_copy = pd.merge(df_copy, df_2_copy, on=['Prefecture', 'Date'])
japan_cases = pd.merge(df_copy, df_3_copy, on=['Date'],how='left')
japan_cases

,Date,Prefecture,Deaths(Cumulative),Newly confirmed cases,PCR 検査実施件数(単日)
0,2020/5/9,Hokkaido,48,11,2265.0
1,2020/5/9,Aomori,0,0,2265.0
2,2020/5/9,Iwate,0,0,2265.0
3,2020/5/9,Miyagi,1,0,2265.0
4,2020/5/9,Akita,0,0,2265.0
...,...,...,...,...,...
24858,2021/10/19,Kumamoto,136,0,51835.0
24859,2021/10/19,Oita,84,0,51835.0
24860,2021/10/19,Miyazaki,41,1,51835.0
24861,2021/10/19,Kagoshima,65,1,51835.0


In [8]:
def show_plot(col1):
    Prefecture =  japan_cases[japan_cases['Prefecture']==col1]
    fig = px.line(Prefecture, x="Date", y=['Deaths(Cumulative)','Newly confirmed cases'])
    fig.update_layout(
      title= col1 + 'の総死者数と新規感染者', 
      font_family="San Serif",
      bargap=0.2,
      barmode='group',
      titlefont={'size': 24},
      paper_bgcolor='#E6E6E6',
      plot_bgcolor='#E6E6E6',                  

      )
    fig.show()
    #return df_copy[df_copy['Prefecture']==col1]

w1 = Select(description='Prefecture:', options=japan_cases['Prefecture'].unique(), rows=4,)

interact(show_plot, col1=w1)

interactive(children=(Select(description='Prefecture:', options=('Hokkaido', 'Aomori', 'Iwate', 'Miyagi', 'Aki…

<function __main__.show_plot(col1)>

In [9]:
Deaths_sum = pd.pivot_table(japan_cases, index='Prefecture', values='Deaths(Cumulative)',margins=False).reset_index()

fig = px.bar(Deaths_sum, x='Prefecture', y='Deaths(Cumulative)', height=600, width=1000, color='Deaths(Cumulative)', color_continuous_scale='mint', template='plotly_white')
fig.update_xaxes(type='category')
fig.update_layout(title={
        'text': "県別の総死者数",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [10]:
japan_cases['Date'] = pd.to_datetime(japan_cases['Date'])
min_date = japan_cases['Date'].min()
japan_cases.groupby('Date').sum()['Deaths(Cumulative)']

Date
2020-05-09      613
2020-05-10      621
2020-05-11      643
2020-05-12      668
2020-05-13      687
              ...  
2021-10-15    18070
2021-10-16    18084
2021-10-17    18095
2021-10-18    18113
2021-10-19    18138
Name: Deaths(Cumulative), Length: 529, dtype: int64

In [11]:
for p in japan_cases['Prefecture'].unique():
    Area = japan_cases[df_copy.iloc[:, 1] == p]
    Diff = Area['Deaths(Cumulative)'].diff()
    #Num = pd.DataFrame(Area['Deaths(Cumulative)'].diff())
    #Num = Num.dropna(how='all').reset_index(drop=True)
    Diffs = np.array(Diff)
    #min_date = df['Date'].min()
    #min_date = '2020/5/9'
    first_ = japan_cases.loc[(japan_cases['Date'] == min_date) & (japan_cases['Prefecture'] == p), 'Deaths(Cumulative)']
    japan_cases.loc[(japan_cases['Date'] == min_date) & (japan_cases['Prefecture'] == p), 'Deaths'] = first_
    
    date = japan_cases['Date'].unique()
    date = np.delete(date,0)
    
    for i, d in enumerate(date, 1):
        japan_cases.loc[(japan_cases['Date'] == d) & (japan_cases['Prefecture'] == p), 'Deaths'] = Diffs[i]
        print(japan_cases.loc[(japan_cases['Date'] == d) & (japan_cases['Prefecture'] == p), 'Deaths'])

47    3.0
Name: Deaths, dtype: float64
94    5.0
Name: Deaths, dtype: float64
141    6.0
Name: Deaths, dtype: float64
188    6.0
Name: Deaths, dtype: float64
235    2.0
Name: Deaths, dtype: float64
282    2.0
Name: Deaths, dtype: float64
329    1.0
Name: Deaths, dtype: float64
376    1.0
Name: Deaths, dtype: float64
423    1.0
Name: Deaths, dtype: float64
470    1.0
Name: Deaths, dtype: float64
517    1.0
Name: Deaths, dtype: float64
564    0.0
Name: Deaths, dtype: float64
611    1.0
Name: Deaths, dtype: float64
658    0.0
Name: Deaths, dtype: float64
705    1.0
Name: Deaths, dtype: float64
752    2.0
Name: Deaths, dtype: float64
799    2.0
Name: Deaths, dtype: float64
846    1.0
Name: Deaths, dtype: float64
893    1.0
Name: Deaths, dtype: float64
940    1.0
Name: Deaths, dtype: float64
987    0.0
Name: Deaths, dtype: float64
1034    0.0
Name: Deaths, dtype: float64
1081    0.0
Name: Deaths, dtype: float64
1128    1.0
Name: Deaths, dtype: float64
1175    0.0
Name: Deaths, dtype: float6

10011    6.0
Name: Deaths, dtype: float64
10058    9.0
Name: Deaths, dtype: float64
10105    16.0
Name: Deaths, dtype: float64
10152    5.0
Name: Deaths, dtype: float64
10199    10.0
Name: Deaths, dtype: float64
10246    6.0
Name: Deaths, dtype: float64
10293    22.0
Name: Deaths, dtype: float64
10340    11.0
Name: Deaths, dtype: float64
10387    11.0
Name: Deaths, dtype: float64
10434    6.0
Name: Deaths, dtype: float64
10481    13.0
Name: Deaths, dtype: float64
10528    10.0
Name: Deaths, dtype: float64
10575    5.0
Name: Deaths, dtype: float64
10622    8.0
Name: Deaths, dtype: float64
10669    7.0
Name: Deaths, dtype: float64
10716    10.0
Name: Deaths, dtype: float64
10763    4.0
Name: Deaths, dtype: float64
10810    8.0
Name: Deaths, dtype: float64
10857    4.0
Name: Deaths, dtype: float64
10904    7.0
Name: Deaths, dtype: float64
10951    5.0
Name: Deaths, dtype: float64
10998    6.0
Name: Deaths, dtype: float64
11045    3.0
Name: Deaths, dtype: float64
11092    5.0
Name: Deaths,

Name: Deaths, dtype: float64
20116    0.0
Name: Deaths, dtype: float64
20163    0.0
Name: Deaths, dtype: float64
20210    1.0
Name: Deaths, dtype: float64
20257    0.0
Name: Deaths, dtype: float64
20304    2.0
Name: Deaths, dtype: float64
20351    1.0
Name: Deaths, dtype: float64
20398    1.0
Name: Deaths, dtype: float64
20445    0.0
Name: Deaths, dtype: float64
20492    1.0
Name: Deaths, dtype: float64
20539    0.0
Name: Deaths, dtype: float64
20586    0.0
Name: Deaths, dtype: float64
20633    1.0
Name: Deaths, dtype: float64
20680    3.0
Name: Deaths, dtype: float64
20727    0.0
Name: Deaths, dtype: float64
20774    0.0
Name: Deaths, dtype: float64
20821    1.0
Name: Deaths, dtype: float64
20868    1.0
Name: Deaths, dtype: float64
20915    0.0
Name: Deaths, dtype: float64
20962    0.0
Name: Deaths, dtype: float64
21009    0.0
Name: Deaths, dtype: float64
21056    1.0
Name: Deaths, dtype: float64
21103    0.0
Name: Deaths, dtype: float64
21150    1.0
Name: Deaths, dtype: float64
21197

6111    0.0
Name: Deaths, dtype: float64
6158    0.0
Name: Deaths, dtype: float64
6205    0.0
Name: Deaths, dtype: float64
6252    0.0
Name: Deaths, dtype: float64
6299    0.0
Name: Deaths, dtype: float64
6346    0.0
Name: Deaths, dtype: float64
6393    0.0
Name: Deaths, dtype: float64
6440    0.0
Name: Deaths, dtype: float64
6487    0.0
Name: Deaths, dtype: float64
6534    0.0
Name: Deaths, dtype: float64
6581    0.0
Name: Deaths, dtype: float64
6628    0.0
Name: Deaths, dtype: float64
6675    0.0
Name: Deaths, dtype: float64
6722    0.0
Name: Deaths, dtype: float64
6769    0.0
Name: Deaths, dtype: float64
6816    0.0
Name: Deaths, dtype: float64
6863    0.0
Name: Deaths, dtype: float64
6910    0.0
Name: Deaths, dtype: float64
6957    0.0
Name: Deaths, dtype: float64
7004    0.0
Name: Deaths, dtype: float64
7051    0.0
Name: Deaths, dtype: float64
7098    0.0
Name: Deaths, dtype: float64
7145    0.0
Name: Deaths, dtype: float64
7192    0.0
Name: Deaths, dtype: float64
7239    0.0
Name

16733    0.0
Name: Deaths, dtype: float64
16780    0.0
Name: Deaths, dtype: float64
16827    1.0
Name: Deaths, dtype: float64
16874    0.0
Name: Deaths, dtype: float64
16921    0.0
Name: Deaths, dtype: float64
16968    1.0
Name: Deaths, dtype: float64
17015    0.0
Name: Deaths, dtype: float64
17062    0.0
Name: Deaths, dtype: float64
17109    2.0
Name: Deaths, dtype: float64
17156    0.0
Name: Deaths, dtype: float64
17203    0.0
Name: Deaths, dtype: float64
17250    0.0
Name: Deaths, dtype: float64
17297    1.0
Name: Deaths, dtype: float64
17344    0.0
Name: Deaths, dtype: float64
17391    0.0
Name: Deaths, dtype: float64
17438    0.0
Name: Deaths, dtype: float64
17485    0.0
Name: Deaths, dtype: float64
17532    0.0
Name: Deaths, dtype: float64
17579    0.0
Name: Deaths, dtype: float64
17626    0.0
Name: Deaths, dtype: float64
17673    0.0
Name: Deaths, dtype: float64
17720    1.0
Name: Deaths, dtype: float64
17767    0.0
Name: Deaths, dtype: float64
17814    0.0
Name: Deaths, dtype: 

...

8552    0.0
Name: Deaths, dtype: float64
8599    0.0
Name: Deaths, dtype: float64
8646    0.0
Name: Deaths, dtype: float64
8693    0.0
Name: Deaths, dtype: float64
8740    0.0
Name: Deaths, dtype: float64
8787    0.0
Name: Deaths, dtype: float64
8834    0.0
Name: Deaths, dtype: float64
8881    0.0
Name: Deaths, dtype: float64
8928    0.0
Name: Deaths, dtype: float64
8975    0.0
Name: Deaths, dtype: float64
9022    0.0
Name: Deaths, dtype: float64
9069    0.0
Name: Deaths, dtype: float64
9116    0.0
Name: Deaths, dtype: float64
9163    0.0
Name: Deaths, dtype: float64
9210    0.0
Name: Deaths, dtype: float64
9257    0.0
Name: Deaths, dtype: float64
9304    0.0
Name: Deaths, dtype: float64
9351    0.0
Name: Deaths, dtype: float64
9398    0.0
Name: Deaths, dtype: float64
9445    0.0
Name: Deaths, dtype: float64
9492    0.0
Name: Deaths, dtype: float64
9539    0.0
Name: Deaths, dtype: float64
9586    0.0
Name: Deaths, dtype: float64
9633    0.0
Name: Deaths, dtype: float64
9680    0.0
Name

18422    0.0
Name: Deaths, dtype: float64
18469    0.0
Name: Deaths, dtype: float64
18516    1.0
Name: Deaths, dtype: float64
18563    0.0
Name: Deaths, dtype: float64
18610    1.0
Name: Deaths, dtype: float64
18657    0.0
Name: Deaths, dtype: float64
18704    1.0
Name: Deaths, dtype: float64
18751    0.0
Name: Deaths, dtype: float64
18798    0.0
Name: Deaths, dtype: float64
18845    0.0
Name: Deaths, dtype: float64
18892    0.0
Name: Deaths, dtype: float64
18939    0.0
Name: Deaths, dtype: float64
18986    0.0
Name: Deaths, dtype: float64
19033    0.0
Name: Deaths, dtype: float64
19080    0.0
Name: Deaths, dtype: float64
19127    1.0
Name: Deaths, dtype: float64
19174    1.0
Name: Deaths, dtype: float64
19221    1.0
Name: Deaths, dtype: float64
19268    2.0
Name: Deaths, dtype: float64
19315    0.0
Name: Deaths, dtype: float64
19362    0.0
Name: Deaths, dtype: float64
19409    0.0
Name: Deaths, dtype: float64
19456    0.0
Name: Deaths, dtype: float64
19503    0.0
Name: Deaths, dtype: 

3618    0.0
Name: Deaths, dtype: float64
3665    0.0
Name: Deaths, dtype: float64
3712    0.0
Name: Deaths, dtype: float64
3759    0.0
Name: Deaths, dtype: float64
3806    0.0
Name: Deaths, dtype: float64
3853    0.0
Name: Deaths, dtype: float64
3900    0.0
Name: Deaths, dtype: float64
3947    0.0
Name: Deaths, dtype: float64
3994    0.0
Name: Deaths, dtype: float64
4041    0.0
Name: Deaths, dtype: float64
4088    0.0
Name: Deaths, dtype: float64
4135    0.0
Name: Deaths, dtype: float64
4182    0.0
Name: Deaths, dtype: float64
4229    0.0
Name: Deaths, dtype: float64
4276    0.0
Name: Deaths, dtype: float64
4323    0.0
Name: Deaths, dtype: float64
4370    0.0
Name: Deaths, dtype: float64
4417    3.0
Name: Deaths, dtype: float64
4464    0.0
Name: Deaths, dtype: float64
4511    0.0
Name: Deaths, dtype: float64
4558    0.0
Name: Deaths, dtype: float64
4605    1.0
Name: Deaths, dtype: float64
4652    0.0
Name: Deaths, dtype: float64
4699    2.0
Name: Deaths, dtype: float64
4746    1.0
Name

13770    0.0
Name: Deaths, dtype: float64
13817    4.0
Name: Deaths, dtype: float64
13864    1.0
Name: Deaths, dtype: float64
13911    0.0
Name: Deaths, dtype: float64
13958    0.0
Name: Deaths, dtype: float64
14005    1.0
Name: Deaths, dtype: float64
14052    0.0
Name: Deaths, dtype: float64
14099    0.0
Name: Deaths, dtype: float64
14146    0.0
Name: Deaths, dtype: float64
14193    0.0
Name: Deaths, dtype: float64
14240    0.0
Name: Deaths, dtype: float64
14287    0.0
Name: Deaths, dtype: float64
14334    0.0
Name: Deaths, dtype: float64
14381    2.0
Name: Deaths, dtype: float64
14428    0.0
Name: Deaths, dtype: float64
14475    0.0
Name: Deaths, dtype: float64
14522    0.0
Name: Deaths, dtype: float64
14569    0.0
Name: Deaths, dtype: float64
14616    0.0
Name: Deaths, dtype: float64
14663    0.0
Name: Deaths, dtype: float64
14710    1.0
Name: Deaths, dtype: float64
14757    0.0
Name: Deaths, dtype: float64
14804    0.0
Name: Deaths, dtype: float64
14851    0.0
Name: Deaths, dtype: 

23875    5.0
Name: Deaths, dtype: float64
23922    0.0
Name: Deaths, dtype: float64
23969    5.0
Name: Deaths, dtype: float64
24016    5.0
Name: Deaths, dtype: float64
24063    5.0
Name: Deaths, dtype: float64
24110    0.0
Name: Deaths, dtype: float64
24157    6.0
Name: Deaths, dtype: float64
24204    4.0
Name: Deaths, dtype: float64
24251    0.0
Name: Deaths, dtype: float64
24298    5.0
Name: Deaths, dtype: float64
24345    4.0
Name: Deaths, dtype: float64
24392    5.0
Name: Deaths, dtype: float64
24439    0.0
Name: Deaths, dtype: float64
24486    4.0
Name: Deaths, dtype: float64
24533    0.0
Name: Deaths, dtype: float64
24580    5.0
Name: Deaths, dtype: float64
24627    6.0
Name: Deaths, dtype: float64
24674    6.0
Name: Deaths, dtype: float64
24721    0.0
Name: Deaths, dtype: float64
24768    0.0
Name: Deaths, dtype: float64
24815    3.0
Name: Deaths, dtype: float64
24862    0.0
Name: Deaths, dtype: float64


In [12]:
japan_cases

,Date,Prefecture,Deaths(Cumulative),Newly confirmed cases,PCR 検査実施件数(単日),Deaths
0,2020-05-09,Hokkaido,48,11,2265.0,48.0
1,2020-05-09,Aomori,0,0,2265.0,0.0
2,2020-05-09,Iwate,0,0,2265.0,0.0
3,2020-05-09,Miyagi,1,0,2265.0,1.0
4,2020-05-09,Akita,0,0,2265.0,0.0
...,...,...,...,...,...,...
24858,2021-10-19,Kumamoto,136,0,51835.0,0.0
24859,2021-10-19,Oita,84,0,51835.0,0.0
24860,2021-10-19,Miyazaki,41,1,51835.0,0.0
24861,2021-10-19,Kagoshima,65,1,51835.0,0.0


In [13]:
confirmed = japan_cases.groupby('Date').sum()['Newly confirmed cases'].reset_index()
deaths = japan_cases.groupby('Date').sum()['Deaths'].reset_index()
pcr = japan_cases.groupby('Date').mean()['PCR 検査実施件数(単日)'].reset_index()


In [14]:
fig = go.Figure()
fig.add_trace(go.Bar(x=confirmed['Date'],
                y=confirmed['Newly confirmed cases'],
                name='Confirmed',
                marker_color='blue'
                ))
fig.add_trace(go.Bar(x=deaths['Date'],
                y=deaths['Deaths'],
                name='Deaths',
                marker_color='Red'
                ))
fig.add_trace(go.Bar(x=pcr['Date'],
                y=pcr['PCR 検査実施件数(単日)'],
                name='PCR 検査実施件数(単日)',
                marker_color='Green'
                ))

fig.update_layout(
    title='Worldwide Corona Virus Cases - Confirmed, Deaths, Recovered (Bar Chart)',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(x=confirmed['Date'],
                y=confirmed['Newly confirmed cases'],
                name='Confirmed',
                marker_color='blue'
                ))
fig.update_layout(
    title='新規感染者の推移',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Bar(x=deaths['Date'],
                y=deaths['Deaths'],
                name='Deaths',
                marker_color='Red'
                ))

fig.update_layout(
    title='コロナによる死者の推移',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [17]:
fig = go.Figure()
fig.add_trace(go.Bar(x=pcr['Date'],
                y=pcr['PCR 検査実施件数(単日)'],
                name='PCR 検査実施件数(単日)',
                marker_color='Green'
                ))

fig.update_layout(
    title='PCR 検査実施件数の推移',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Cases',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

In [18]:
prefecture_cases = japan_cases.groupby('Prefecture')['Newly confirmed cases','Deaths'].max().reset_index()
prefecture_cases["Death Rate"] = np.round(100*prefecture_cases["Deaths"]/prefecture_cases["Newly confirmed cases"],2)
state_wise_cases=prefecture_cases.sort_values('Newly confirmed cases', ascending= False).fillna(0).style.background_gradient(cmap='Blues',subset=["Newly confirmed cases"]).background_gradient(cmap='Reds',subset=["Deaths"]).background_gradient(cmap='RdYlBu',subset=["Death Rate"])
state_wise_cases

,Prefecture,Newly confirmed cases,Deaths,Death Rate
40,Tokyo,5773,180.000000,3.120000
32,Osaka,3004,59.000000,1.960000
18,Kanagawa,2878,47.000000,1.630000
0,Aichi,2346,34.000000,1.450000
34,Saitama,2169,41.000000,1.890000
3,Chiba,1778,39.000000,2.190000
6,Fukuoka,1253,24.000000,1.920000
12,Hyogo,1088,129.000000,11.860000
31,Okinawa,809,7.000000,0.870000
11,Hokkaido,727,48.000000,6.600000


# 致命率 case fatality rate

In [76]:
japan_cases['Fatality-Ratio'] = 100*np.round(japan_cases["Deaths"]/japan_cases["Newly confirmed cases"],2)
japan_cases['Fatality-Ratio'] = japan_cases['Fatality-Ratio'].replace([np.inf, -np.inf], np.nan)
japan_cases['Fatality-Ratio'] = japan_cases['Fatality-Ratio'].fillna(0)
fatality_ratio=japan_cases.groupby('Prefecture')['Fatality-Ratio'].mean().reset_index()

In [78]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=fatality_ratio.Prefecture, y=fatality_ratio['Fatality-Ratio'], name="Fatality-Ratio",line_shape='linear'))

fig.update_traces(mode='lines+markers')
fig.update_layout(title='都道府県別 致命率',legend=dict(y=0.5, traceorder='reversed', font_size=16))

fig.show()

In [85]:
fatality_year=japan_cases.groupby('Date')['Fatality-Ratio'].mean().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x=fatality_year.Date, y=fatality_year['Fatality-Ratio'], name="Fatality-Ratio",line_shape='linear'))

fig.update_layout(title='都道府県別 致命率',legend=dict(y=0.5, traceorder='reversed', font_size=16))

fig.show()

2020年5月9日の数値が異様に高いのは死者の人数が600人から始まる為
その後は、徐々に増加する。

In [89]:
japan_cases.groupby('Date').sum()['Deaths(Cumulative)']

Date
2020-05-09      613
2020-05-10      621
2020-05-11      643
2020-05-12      668
2020-05-13      687
              ...  
2021-10-15    18070
2021-10-16    18084
2021-10-17    18095
2021-10-18    18113
2021-10-19    18138
Name: Deaths(Cumulative), Length: 529, dtype: int64

In [102]:
top_10 = japan_cases.groupby('Prefecture')['Newly confirmed cases'].max().sort_values(ascending=False).reset_index()

trace = go.Table(
    domain=dict(x=[0, 0.52],
                y=[0, 1.0]),
    header=dict(values=["Prefecture","Newly confirmed cases"],
                fill = dict(color = '#119DFF'),
                font = dict(color = 'white', size = 14),
                align = ['center'],
               height = 30),
    cells=dict(values=[top_10['Prefecture'].head(10),top_10['Newly confirmed cases'].head(10)],
               fill = dict(color = ['#DDD', 'white']),
               align = ['center'],height=20))

trace1 = go.Bar(x=top_10['Prefecture'].head(10),
                y=top_10['Newly confirmed cases'].head(10),
                xaxis='x1',
                yaxis='y1',
                marker=dict(color='blue'),opacity=0.60)
layout = dict(
    width=1000,
    height=400,
    autosize=False,
    title='Top-10 新規感染者が多い県名',
    showlegend=False,   
    xaxis1=dict(**dict(domain=[0.58, 1], anchor='y1', showticklabels=True)),
    yaxis1=dict(**dict(domain=[0, 1.0], anchor='x1', hoverformat='.2f')),  
)
fig1 = dict(data=[trace, trace1], layout=layout)
iplot(fig1)

In [104]:
top_10 = japan_cases.groupby('Prefecture')['Deaths'].sum().sort_values(ascending=False).reset_index()

trace = go.Table(
    domain=dict(x=[0, 0.52],
                y=[0, 1.0]),
    header=dict(values=["Prefecture","Deaths"],
                fill = dict(color = '#119DFF'),
                font = dict(color = 'white', size = 14),
                align = ['center'],
               height = 30),
    cells=dict(values=[top_10['Prefecture'].head(10),top_10['Deaths'].head(10)],
               fill = dict(color = ['#DDD', 'white']),
               align = ['center'],height=20))

trace1 = go.Bar(x=top_10['Prefecture'].head(10),
                y=top_10['Deaths'].head(10),
                xaxis='x1',
                yaxis='y1',
                marker=dict(color='blue'),opacity=0.60)
layout = dict(
    width=1000,
    height=400,
    autosize=False,
    title='Top-10 死者数が多い県名',
    showlegend=False,   
    xaxis1=dict(**dict(domain=[0.58, 1], anchor='y1', showticklabels=True)),
    yaxis1=dict(**dict(domain=[0, 1.0], anchor='x1', hoverformat='.2f')),  
)
fig1 = dict(data=[trace, trace1], layout=layout)
iplot(fig1)